Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

AttributeError: module 'tensorflow' has no attribute 'placeholder'

In [ ]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0 : 3.51275
Minibatch accuracy: 6.2%
Validation accuracy: 12.8%
Minibatch loss at step 50 : 1.48703
Minibatch accuracy: 43.8%
Validation accuracy: 50.4%
Minibatch loss at step 100 : 1.04377
Minibatch accuracy: 68.8%
Validation accuracy: 67.4%
Minibatch loss at step 150 : 0.601682
Minibatch accuracy: 68.8%
Validation accuracy: 73.0%
Minibatch loss at step 200 : 0.898649
Minibatch accuracy: 75.0%
Validation accuracy: 77.8%
Minibatch loss at step 250 : 1.3637
Minibatch accuracy: 56.2%
Validation accuracy: 75.4%
Minibatch loss at step 300 : 1.41968
Minibatch accuracy: 62.5%
Validation accuracy: 76.0%
Minibatch loss at step 350 : 0.300648
Minibatch accuracy: 81.2%
Validation accuracy: 80.2%
Minibatch loss at step 400 : 1.32092
Minibatch accuracy: 56.2%
Validation accuracy: 80.4%
Minibatch loss at step 450 : 0.556701
Minibatch accuracy: 81.2%
Validation accuracy: 79.4%
Minibatch loss at step 500 : 1.65595
Minibatch accuracy: 43.8%
Validation accuracy: 79.6%

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [25]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16,(5,5),input_shape=(28,28,1),activation='relu',padding='same'))
model.add(tf.keras.layers.MaxPool2D(2,strides=2))
model.add(tf.keras.layers.Conv2D(16,(5,5),activation='relu',padding='same'))
model.add(tf.keras.layers.MaxPool2D(2,strides=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(784,activation='relu'))
model.add(tf.keras.layers.Dense(10))

loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',loss=loss_fn,metrics=['accuracy'])

In [27]:
history = model.fit(train_dataset,train_labels,validation_data=(valid_dataset,valid_labels),epochs=25)

Epoch 1/25
6250/6250 [==============================] - 34s 5ms/step - loss: 0.2526 - accuracy: 0.9224 - val_loss: 0.2683 - val_accuracy: 0.9215
Epoch 2/25
6250/6250 [==============================] - 36s 6ms/step - loss: 0.2065 - accuracy: 0.9356 - val_loss: 0.2685 - val_accuracy: 0.9227
Epoch 3/25
6250/6250 [==============================] - 38s 6ms/step - loss: 0.1662 - accuracy: 0.9479 - val_loss: 0.2742 - val_accuracy: 0.9251
Epoch 4/25
6250/6250 [==============================] - 42s 7ms/step - loss: 0.1298 - accuracy: 0.9590 - val_loss: 0.3109 - val_accuracy: 0.9267
Epoch 5/25
6250/6250 [==============================] - 43s 7ms/step - loss: 0.1032 - accuracy: 0.9675 - val_loss: 0.3421 - val_accuracy: 0.9227
Epoch 6/25
6250/6250 [==============================] - 43s 7ms/step - loss: 0.0832 - accuracy: 0.9742 - val_loss: 0.3878 - val_accuracy: 0.9260
Epoch 7/25
6250/6250 [==============================] - 45s 7ms/step - loss: 0.0705 - accuracy: 0.9785 - val_loss: 0.4380 - val_ac

KeyboardInterrupt: 

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [49]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16,(5,5),input_shape=(28,28,1),activation='relu',padding='valid'))
model.add(tf.keras.layers.MaxPool2D(2,strides=2))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Conv2D(16,(5,5),activation='relu',padding='valid'))
model.add(tf.keras.layers.MaxPool2D(2,strides=2))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256,activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10))

loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',loss=loss_fn,metrics=['accuracy'])

In [50]:
model.summary()

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_39 (Conv2D)          (None, 24, 24, 16)        416       
                                                                 
 max_pooling2d_34 (MaxPoolin  (None, 12, 12, 16)       0         
 g2D)                                                            
                                                                 
 dropout_20 (Dropout)        (None, 12, 12, 16)        0         
                                                                 
 conv2d_40 (Conv2D)          (None, 8, 8, 16)          6416      
                                                                 
 max_pooling2d_35 (MaxPoolin  (None, 4, 4, 16)         0         
 g2D)                                                            
                                                                 
 dropout_21 (Dropout)        (None, 4, 4, 16)        

In [ ]:
history = model.fit(train_dataset,train_labels,validation_data=(valid_dataset,valid_labels),epochs=25)

Epoch 1/25
6250/6250 [==============================] - 25s 4ms/step - loss: 0.7624 - accuracy: 0.7678 - val_loss: 0.4616 - val_accuracy: 0.8610
Epoch 2/25
6250/6250 [==============================] - 25s 4ms/step - loss: 0.5875 - accuracy: 0.8253 - val_loss: 0.4275 - val_accuracy: 0.8686
Epoch 3/25
6250/6250 [==============================] - 26s 4ms/step - loss: 0.5617 - accuracy: 0.8340 - val_loss: 0.4253 - val_accuracy: 0.8718
Epoch 4/25
6250/6250 [==============================] - 25s 4ms/step - loss: 0.5448 - accuracy: 0.8382 - val_loss: 0.3953 - val_accuracy: 0.8781
Epoch 5/25
6250/6250 [==============================] - 25s 4ms/step - loss: 0.5388 - accuracy: 0.8404 - val_loss: 0.4023 - val_accuracy: 0.8779
Epoch 6/25
6250/6250 [==============================] - 25s 4ms/step - loss: 0.5334 - accuracy: 0.8421 - val_loss: 0.3896 - val_accuracy: 0.8854
Epoch 7/25
6250/6250 [==============================] - 26s 4ms/step - loss: 0.5302 - accuracy: 0.8432 - val_loss: 0.3912 - val_ac